In [11]:
import pandas as pd 
import os
from pathlib import Path
from tqdm import tqdm 
import os
from pathlib import Path
import pandas as pd
from tqdm import tqdm
from typing import Optional

### utils


In [16]:
def min_to_second(time_str: Optional[str]) -> int:
    """
    Перевод времени в удобный формат (в секунды)
    Аргументы:
       time_str - время в формате str 
    Возвращает:
        время в секундах
    """
    
    if isinstance(time_str, str):
        minute, second =time_str.split(':')
        minute = int(minute.split('.')[0]) * 60 
        second = int(second)
        return  second + minute
    return time_str

def search_signal_indicators(path_resource: str, player_id:int, targets: list[str]):
    """
    Рассчёт показателей (targets) для игрока с указанным id для всех игр сезона 
    
    Аргументы:
       path_resource - путьдо папки с файлами для каждой игры сезона 
       player_id - идентификатор игрока 
       targets - целевые переменные, которые необходимо отслеживать
    """
    file_list = os.listdir(path_resource) # список путей для всех файлов с играми сезона 
    all_stat = pd.Series()
    for file_name in tqdm(file_list, total=len(file_list)):
            df = pd.read_csv(Path(path_resource) / Path(file_name))
            if  'PLAYER_ID' not in df.columns:
                continue
            elif df['PLAYER_ID'].isin([player_id]).any():
                one_player = df.query(f"""PLAYER_ID=={player_id}""")[targets].iloc[0,:]
                all_stat  = pd.concat([all_stat , one_player], axis=1).fillna(0) 
                
                

    all_stat = all_stat.fillna(0).T.rename(columns={'MIN':'SECOND'})    
    all_stat['MIN'] = all_stat['MIN'].apply(min_to_second)
    all_stat = all_stat.sum(0)
    all_stat['PLAYER_ID'] = player_id
    
    return all_stat
    

### get stat

In [18]:

"""
- 'MIN': время на площадке (в минутах)
- 'FGM': количество успешных попыток бросков из игры
- 'FGA': количество попыток бросков из игры
- 'FG3M': количество успешных трехочковых бросков
- 'FG3A': количество попыток трехочковых бросков
- 'FTM': количество успешных штрафных бросков
- 'FTA': количество попыток штрафных бросков
- 'FT_PCT': процент успешных штрафных бросков
- 'OREB': количество подборов в нападении
- 'DREB': количество подборов в защите
- 'REB': общее количество подборов
- 'AST': количество передач
- 'STL': количество перехватов
- 'BLK': количество блок-шотов
- 'TO': количество потерь мяча
- 'PF': количество фолов
- 'PTS': количество набранных очков
"""

target_columns = ['MIN', 'FGM', 'FGA', 'FG3M','FTM','FTA','FT_PCT',  'OREB','DREB','REB','AST','STL','BLK', 'TO','PF','PTS']
folder_path = r'resource\boxscoretraditionalv2'  # Замените на путь к вашей папке c файлами 
file_list = os.listdir(folder_path)
player_id = 1628975

search_signal_indicators(path_resource=folder_path,
                         player_id=player_id, 
                         targets=target_columns)

  0%|          | 0/6273 [00:00<?, ?it/s]

 13%|█▎        | 836/6273 [00:02<00:13, 404.32it/s]

In [8]:
aggregation_data = all_stat.sum(0)
aggregation_data

MIN       67384
FGM       123.0
FGA       308.0
FG3M       80.0
FTM        17.0
FTA        20.0
FT_PCT      7.5
OREB       18.0
DREB      114.0
REB       132.0
AST       111.0
STL        35.0
BLK        15.0
TO         47.0
PF         89.0
PTS       343.0
dtype: object